# Importing all the required packages

In [ ]:

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

#importing the dataset 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        




# Obtaining and pre-processing of data

In [ ]:
wine = pd.read_csv("/kaggle/input/wine-quality/winequalityN.csv")
wine.head()
wine.info()

In [ ]:
wine.shape

In [ ]:
wine.isnull().sum()

In [ ]:
wine = wine.dropna(axis=0)

In [ ]:
wine.isnull().sum()

In [ ]:
wine.duplicated().sum()

In [ ]:
wine.drop_duplicates(keep='first', inplace=True)

In [ ]:
count_cat = wine.nunique()

In [ ]:
count_cat

In [ ]:
dis = wine['type'].value_counts(normalize = True)

In [ ]:
dis

In [ ]:
plt.figure(figsize = (5,5))
sns.countplot(x= wine['type'], palette =['#808080', '#722F37'])

In [ ]:
labels = ['White', 'Red']
sizes = [wine['type'].value_counts()[0],wine['type'].value_counts()[1]]
col = ['#808080', '#722F37']
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, colors=col, autopct = '%1.1f%%')
ax1.axis('equal')
plt.tight_layout()
plt.show()


In [ ]:
encoder = LabelEncoder()
encoder.fit(wine['type'])
list(encoder.classes_)
wine['class']=wine['type']
wine['type']=encoder.transform(wine['type'])





# Exploaratory Data Analysis

In [ ]:
wine.head()

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(wine.corr(), square=True, annot=True, cmap='Purples')

As you can see, quality and alcohol are very weakly correlated to type.
So we're going to drop them from our feature set.

In [ ]:
wine.groupby('class')['total sulfur dioxide'].mean().to_frame(name='mean').reset_index()

In [ ]:
wine.groupby('class')['total sulfur dioxide'].mean().plot.bar(x='Type',y='mean',color = ["#722F37","#808080"] )

In [ ]:
boxdata1 = wine[wine['type']==0]
boxdata1 = boxdata1[['total sulfur dioxide']]

plt.boxplot(boxdata1)
plt.show()



In [ ]:
boxdata2 = wine[wine['type']==1]
boxdata2 = boxdata2[['total sulfur dioxide']]

plt.boxplot(boxdata2)
plt.show()



In [ ]:
wine.groupby('class')['free sulfur dioxide'].mean().plot.bar(x='Type',y='mean', color =['#722F37', '#808080'])

In [ ]:
wine.groupby('class')['residual sugar'].mean().plot.bar(x='Type',y='mean', color =['#722F37', '#808080'])

In [ ]:
wine.groupby('class')['volatile acidity'].mean().plot.bar(x='Type',y='mean', color =['#722F37', '#808080'])

In [ ]:
wine.groupby('class')['chlorides'].mean().plot.bar(x='Type',y='mean', color =['#722F37', '#808080'])

# **Scaling**

In [ ]:
wine.describe()

In [ ]:
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates']
ss = StandardScaler()
wine.drop('class', inplace=True, axis =1)
df = wine
df[features]= ss.fit_transform(df[features])
df.head()

# **Splitting of Data**

In [ ]:
target = df['type']
data = df.drop(['type'], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(data, target, random_state = 11)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
Y_train.shape

In [ ]:
Y_test.shape

# **Dummy Classifier**

In [ ]:
dm = DummyClassifier(random_state = 11, strategy = 'most_frequent')
dm.fit(X_train, Y_train)

In [ ]:
Y_pred = dm.predict(X_test)

In [ ]:
print(classification_report(Y_test, Y_pred))

In [ ]:
cm_dm = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cm_dm, annot = True, annot_kws = {'size':15}, fmt=".0f")
plt.xlabel("Predict")
plt.ylabel("Actual")

In [ ]:
fpr, tpr, threshholds= roc_curve(Y_test, Y_pred)
auc = roc_auc_score(Y_test, Y_pred)
plt.plot(fpr, tpr, label="auc="+str(auc), color = 'r')
plt.title('ROC Curve Dummy Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.grid(True)
plt.show()
dummy_auc = round(auc,3)*100
print(f"The score for the ROC Curve is: {round(auc, 3)*100}%")


# **KNN - K-Nearest Neighbours**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, Y_train)
knn

In [ ]:
Y_pred = knn.predict(X_test)

In [ ]:
print(classification_report(Y_test, Y_pred))

# Hyperparameter Tuning

In [ ]:
k_range = list(range(1,51))
weight_op = ['uniform', 'distance']
d = {'n_neighbors':k_range, 'weights': weight_op}

In [ ]:
grid_temp = GridSearchCV(knn, d, cv=5, scoring='accuracy')
grid_temp.fit(data, target)
print("score:" ,grid_temp.best_score_, "params:" ,grid_temp.best_params_)

# Evaluation

In [ ]:
fpr, tpr, thresholds= roc_curve(Y_test, Y_pred)
auc = roc_auc_score(Y_test, Y_pred)
plt.plot(fpr, tpr, label="auc="+str(auc), color = 'r')
plt.title('ROC Curve Dummy Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.grid(True)
plt.show()
dummy_auc = round(auc,3)*100
print(f"The score for the ROC Curve is: {round(auc, 3)*100}%")



# RNN - Radius Neighbors Classifier 

In [ ]:
rnn = RadiusNeighborsClassifier( radius = 3.0, weights = 'distance', algorithm ='auto', outlier_label = 'most_frequent')
rnn.fit(X_train, Y_train)
rnn

In [ ]:
Y_pred = rnn.predict(X_test)

In [ ]:
print(classification_report(Y_test, Y_pred))

# Hyperparameter Tuning

In [ ]:
r_range = list(range(1,31))
d = {'radius': r_range}


In [ ]:
grid_temp = GridSearchCV(rnn, d, cv=5, scoring='accuracy')
grid_temp.fit(data, target)
print("score:" ,grid_temp.best_score_, "params:" ,grid_temp.best_params_)

# Evaluation

In [ ]:
cm_dm = confusion_matrix(Y_test, Y_pred)
sns.heatmap(cm_dm, annot = True, annot_kws = {'size':15}, fmt=".0f")
plt.xlabel("Predict")
plt.ylabel("Actual")

In [ ]:
fpr, tpr, thresholds= roc_curve(Y_test, Y_pred)
auc = roc_auc_score(Y_test, Y_pred)
plt.plot(fpr, tpr, label="auc="+str(auc), color = 'r')
plt.title('ROC Curve Dummy Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.grid(True)
plt.show()
dummy_auc = round(auc,3)*100
print(f"The score for the ROC Curve is: {round(auc, 3)*100}%")




# **Logistic Regression**

In [ ]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, Y_train)
predictions = lr.predict(X_test)

In [ ]:
print(classification_report(Y_test, predictions))

# Evaluation

In [ ]:
cm = confusion_matrix(Y_test, predictions)
sns.heatmap(cm, annot=True, annot_kws = {'size': 15}, fmt='.0f')
plt.xlabel("Predict")
plt.ylabel("Actual")

In [ ]:
print(metrics.accuracy_score(Y_test, predictions))

In [ ]:
fpr, tpr, thresholds= roc_curve(Y_test, predictions)
auc = roc_auc_score(Y_test, predictions)
plt.plot(fpr, tpr, label="auc="+str(auc), color = 'r')
plt.title('ROC Curve Dummy Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.grid(True)
plt.show()
dummy_auc = round(auc,3)*100
print(f"The score for the ROC Curve is: {round(auc, 3)*100}%")




# The score for the ROC Curve for each algorithm

**K-Nearest Neighbors - 98.8 <br>
Radius Nearest Neighbors - 96.1 <br>
Logistic Regression - 98.6**